In [1]:
import pandas as pd
import numpy as np  
import re 
import string

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")

In [4]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/theDoctor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
from textblob import TextBlob

In [24]:
def load_dataset(filepath):
    df = pd.read_csv(filepath, encoding='latin-1')
    df.columns = ["text","sentiment"]
    return df

In [22]:
data = []
def load_text_data(filename):
    textfile = open(filename, "r")
    for tweet in textfile:
        print(tweet)
        parts = tweet.split(',')
        if len(parts) > 2:
            newTweet = [parts[0],parts[1], 0, "".join(parts[3:])]
            data.append(newTweet)
    textfile.close()
    df = pd.DataFrame(data)
    df.columns = ["User", "Date", "Retweet", "text"]
    return df


In [6]:
def delete_redundant_cols(df, cols):
    for col in cols:
        del df[col]
        return df

In [27]:
def preprocess_tweet_text(tweet):

    tweet = tweet.lower()

    tweet = re.sub(r"http\S+|www\S+|https\S+","",tweet, flags=re.MULTILINE)   

    tweet = tweet.translate(str.maketrans("","",string.punctuation))

    tweet = re.sub(r'\@\w+|\#',"",tweet)

    tweet_tokens = word_tokenize(tweet)
    filtered_words = [word for word in tweet_tokens if word not in stop_words]

    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]

    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]

    return " ".join(lemma_words)
preprocess_tweet_text("Hi there, how are you preparing for your exams?")    

'hi prepar exam'

In [8]:
def get_feature_vector(train_fit):
    vector = TfidfVectorizer(sublinear_tf=True)
    vector.fit(train_fit)
    return vector

In [50]:
def int_to_string(sentiment):
    if sentiment < -0.1:
        return "Negative"
    elif sentiment < 0.1:
        return "Neutral"
    else:
        return "Positive"

In [71]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676 entries, 0 to 675
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   User       676 non-null    object 
 1   Date       676 non-null    object 
 2   Retweet    676 non-null    int64  
 3   text       676 non-null    object 
 4   cleanText  676 non-null    object 
 5   sentiment  676 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 31.8+ KB


In [83]:
# Load dataset
dataset = load_text_data("datasets/dataset 2021090410_24_02.txt")
# Remove unwanted columns from dataset
# n_dataset = remove_unwanted_cols(dataset, ['t_id', 'created_at', 'query', 'user'])
#Preprocess data
dataset["cleanText"] = dataset['text'].apply(preprocess_tweet_text)

sent = []

for x in dataset['cleanText']:
    s = TextBlob(x).sentiment.polarity
    if s < -0.1:
        r = -1
    elif s > 0.1:
        r = 1
    else:
        r = 0
    sent.append(r) 
            



dataset["sentiment"] = sent



# Split dataset into Train, Test

# Same tf vector will be used for Testing sentiments on unseen trending data
tf_vector = get_feature_vector(np.array(dataset['cleanText']).ravel())
X = tf_vector.transform(np.array(dataset['cleanText']).ravel())
y = np.array(dataset['sentiment']).ravel()
# y = y.astype('float')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

# Training Naive Bayes model
NB_model = MultinomialNB()
NB_model.fit(X_train, y_train)
y_predict_nb = NB_model.predict(X_test)
print(accuracy_score(y_test, y_predict_nb))

# Training Logistics Regression model
LR_model = LogisticRegression(solver='lbfgs')
LR_model.fit(X_train, y_train)
y_predict_lr = LR_model.predict(X_test)
print(accuracy_score(y_test, y_predict_lr))

HarunaMakoleA, Sat Sep 04 15:22:51 +0000 2021, False, @NomthandazoPe11 #happy Sunday too beautiful 🖐🖐

beyondsummitllc, Sat Sep 04 15:21:57 +0000 2021, False, Save more… play more!  Our entire store is discounted until 9/6/21.  #love #instagood #photooftheday #beautiful… https://t.co/mm7Z4hM55s

Rohitwavhal1, Sat Sep 04 15:21:40 +0000 2021, False, @AnviFightsSMA @Vikramtek @aajtak Please help us to save Anvi 🙏🏻



#anvifightssma #wesupportanvi #anviwavhal… https://t.co/WZFP2vlV4R

PeterLamya, Sat Sep 04 15:21:20 +0000 2021, False, We could never have loved the earth so well if we had had no sisterhood in it.

#PeterLamyaPhotography… https://t.co/0BSHWQmrqA

jusdax, Sat Sep 04 15:21:02 +0000 2021, False, if you have the ability to love . Love yourself first.

#adityabruh #adityaforever #jusdaxbruh #cheekfiller #fillers… https://t.co/c4zY0ByqFf

pawzpace, Sat Sep 04 15:16:36 +0000 2021, False, Experience the heist on @pawzpace .... Get up to 60% off on all the grooming services.



For m

1.0
1.0


In [87]:
dataset[dataset['sentiment'] < 0]

,User,Date,Retweet,text,cleanText,sentiment
3,PeterLamya,Sat Sep 04 15:21:20 +0000 2021,0,We could never have loved the earth so well i...,could never love earth well sisterhood,-1
33,pieterstaaks,Sat Sep 04 14:52:10 +0000 2021,0,little happy friend he only likes me because ...,littl happi friend like ill throw skippybal 🐕,-1
41,HanByeol_91,Sat Sep 04 14:43:29 +0000 2021,0,If at least you’re happy\n,least ’ happi,-1
55,PeterLamya,Sat Sep 04 15:21:20 +0000 2021,0,We could never have loved the earth so well i...,could never love earth well sisterhood,-1
85,pieterstaaks,Sat Sep 04 14:52:10 +0000 2021,0,little happy friend he only likes me because ...,littl happi friend like ill throw skippybal 🐕,-1
...,...,...,...,...,...,...
1073,pieterstaaks,Sat Sep 04 14:52:10 +0000 2021,0,little happy friend he only likes me because ...,littl happi friend like ill throw skippybal 🐕,-1
1081,HanByeol_91,Sat Sep 04 14:43:29 +0000 2021,0,If at least you’re happy\n,least ’ happi,-1
1095,PeterLamya,Sat Sep 04 15:21:20 +0000 2021,0,We could never have loved the earth so well i...,could never love earth well sisterhood,-1
1125,pieterstaaks,Sat Sep 04 14:52:10 +0000 2021,0,little happy friend he only likes me because ...,littl happi friend like ill throw skippybal 🐕,-1


In [ ]:
# test_file_name = "trending_tweets/08-04-2020-1586291553-tweets.csv"
# test_ds = load_dataset(test_file_name, ["t_id", "hashtag", "created_at", "user", "text"])
# test_ds = remove_unwanted_cols(test_ds, ["t_id", "created_at", "user"])

# # Creating text feature
# test_ds.text = test_ds["text"].apply(preprocess_tweet_text)
# test_feature = tf_vector.transform(np.array(test_ds.iloc[:, 1]).ravel())

# # Using Logistic Regression model for prediction
# test_prediction_lr = LR_model.predict(test_feature)

# # Averaging out the hashtags result
# test_result_ds = pd.DataFrame({'hashtag': test_ds.hashtag, 'prediction':test_prediction_lr})
# test_result = test_result_ds.groupby(['hashtag']).max().reset_index()
# test_result.columns = ['heashtag', 'predictions']
# test_result.predictions = test_result['predictions'].apply(int_to_string)

# print(test_result)